In [1]:
import pandas as pd
import re
import os
import fitz  # PyMuPDF
# In notebooks/Colab, install BEFORE importing fitz on a fresh runtime:
# !pip install PyMuPDF
import spacy
from transformers import pipeline

/Users/rezafayaz/Documents/Dissertation MDS /Final Files/BigOilNLPAnalysis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/rezafayaz/Documents/Dissertation MDS /Final Files/BigOilNLPAnalysis/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# data extraction

def filename_creator(sus_report):
    """Extract company name and year from filename"""
    name = os.path.splitext(os.path.basename(sus_report))[0]
    parts = re.split(r"[_\-\s]", name)
    year = next((p for p in parts if re.fullmatch(r"\d{4}", p)), "Unknown")
    companies = {"BP", "TOTAL", "SHELL","EXXON","CHEVRON"}  # extend later
    company = next((p.upper() for p in parts if p.upper() in companies), "Unknown")
    return company, year

def extract_pages_fitz(file_path, sparse_threshold=50):
    """
    Extract text from PDF using PyMuPDF with proper page numbers.
    Keeps ALL pages (including short ones), and flags sparse pages.
    """
    data = []

    try:
        doc = fitz.open(file_path)

        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text("text").strip()  # Extract + trim

            char_count = len(text)
            word_count = len(text.split()) if text else 0

            data.append({
                "page_number": page_num + 1,  # 1-indexed
                "text": text,
                "char_count": char_count,
                "word_count": word_count,
                "is_sparse": char_count < sparse_threshold
            })

        doc.close()

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

    return data

def process_all_pdfs(folder_path, sparse_threshold=50):
    """Process all PDFs in folder and create dataset"""
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            print(f"📄 Processing: {filename}")

            company, year = filename_creator(filename)
            pages_data = extract_pages_fitz(file_path, sparse_threshold=sparse_threshold)

            for page_info in pages_data:
                all_data.append({
                    "Company": company,
                    "Year": year,
                    "Page": page_info["page_number"],
                    "Text": page_info["text"],
                    "Char_Count": page_info["char_count"],
                    "Word_Count": page_info["word_count"],
                    "Is_Sparse": page_info["is_sparse"],
                })

    return pd.DataFrame(all_data)

# Usage
folder_path = "content/"
df = process_all_pdfs(folder_path, sparse_threshold=50)

# Save to CSV
df.to_csv("Datasets/BIGOIL_Original_Dataset.csv", index=False)
print(f"✅ Extracted {len(df)} pages from {df['Company'].nunique()} companies")
df.head()


📄 Processing: SHELL_CSR_2022.pdf
📄 Processing: TOTAL_CSR_2022.pdf
📄 Processing: TOTAL_CSR_2023.pdf
📄 Processing: SHELL_CSR_2023.pdf
📄 Processing: BP_CSR_2017.pdf
📄 Processing: SHELL_CSR_2021.pdf
📄 Processing: TOTAL_CSR_2021.pdf
📄 Processing: CHEVRON_CSR_2017.pdf
📄 Processing: TOTAL_CSR_2020.pdf
📄 Processing: SHELL_CSR_2020.pdf
📄 Processing: SHELL_CSR_2018.pdf
📄 Processing: TOTAL_CSR_2024.pdf
📄 Processing: TOTAL_CSR_2018.pdf
📄 Processing: TOTAL_CSR_2019.pdf
📄 Processing: SHELL_CSR_2019.pdf
📄 Processing: EXXON_CSR_2018.pdf
📄 Processing: EXXON_CSR_2024.pdf
📄 Processing: EXXON_CSR_2019.pdf
📄 Processing: EXXON_CSR_2022.pdf
📄 Processing: EXXON_CSR_2023.pdf
📄 Processing: EXXON_CSR_2017.pdf
📄 Processing: BP_CSR_2021.pdf
📄 Processing: SHELL_CSR_2017.pdf
📄 Processing: TOTAL_CSR_2017.pdf
📄 Processing: CHEVRON_CSR_2021.pdf
📄 Processing: CHEVRON_CSR_2020.pdf
📄 Processing: BP_CSR_2020.pdf
📄 Processing: BP_CSR_2022.pdf
📄 Processing: CHEVRON_CSR_2022.pdf
📄 Processing: CHEVRON_CSR_2023.pdf
📄 Processing

,Company,Year,Page,Text,Char_Count,Word_Count,Is_Sparse
0,SHELL,2022,1,Responsible \nenergy\n#PoweringProgress\nShell...,76,8,False
1,SHELL,2022,2,Contents\nRead the Shell Sustainability Report...,1876,271,False
2,SHELL,2022,3,Sustainability at Shell\nWelcome to the Shell ...,510,74,False
3,SHELL,2022,4,Letter from the CEO\nWael Sawan\nChief Executi...,4180,669,False
4,SHELL,2022,5,Respecting nature\nPowering Progress is also a...,3349,531,False


In [ ]:
# chunking the writing blocks

df = pd.read_csv("Datasets/BIGOIL_Original_Dataset.csv")

nlp = spacy.load("en_core_web_sm", disable=["ner", "tagger"])

def split_long_text(
    text,
    target_words=400,
    max_words=450
):
    """
    Split text into sentence-based chunks.
    Sentences are accumulated until ~target_words.
    """
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]

    chunks = []
    current_chunk = []
    current_word_count = 0

    for sent in sentences:
        sent_word_count = len(sent.split())

        if current_word_count + sent_word_count <= target_words:
            current_chunk.append(sent)
            current_word_count += sent_word_count
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sent]
            current_word_count = sent_word_count

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

new_rows = []

for _, row in df.iterrows():
    text = str(row["Text"])
    word_count = row["Word_Count"]

    # Case 1: short rows → keep as-is
    if word_count <= 450:
        new_row = row.to_dict()
        new_row["Subpage"] = float(row["Page"])  # e.g. 40.0
        new_rows.append(new_row)
        continue

    # Case 2: long rows → split
    chunks = split_long_text(text)

    for i, chunk in enumerate(chunks, start=1):
        new_row = row.to_dict()
        new_row["Text"] = chunk
        new_row["Word_Count"] = len(chunk.split())
        new_row["Char_Count"] = len(chunk)
        new_row["Subpage"] = float(f"{int(row['Page'])}.{i}")  # 40.1, 40.2, ...
        new_rows.append(new_row)

df_chunked = pd.DataFrame(new_rows)

print("Original rows:", len(df))
print("New rows after chunking:", len(df_chunked))

df_chunked.to_csv(
    "Datasets/BIGOIL_Chuncked_Dataset.csv",
    index=False
)

print("✅ Saved: BIGOIL_pages_sentence_chunked.csv")



/Users/rezafayaz/Documents/Dissertation MDS /Final Files/BigOilNLPAnalysis/.venv/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


Original rows: 4849
New rows after chunking: 9038
✅ Saved: BIGOIL_pages_sentence_chunked.csv


In [ ]:
# Filtering and clearing
# 1. Load CHUNKED dataset (important)
df = pd.read_csv("Datasets/BIGOIL_Chuncked_Dataset.csv")

# 3. Light cleaning
def light_clean(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', str(text))
    text = text.replace('\n', ' ')
    text = text.replace('-', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

df["Text"] = df["Text"].apply(light_clean)

# 4. Green / environmental lexicon
greenwashing_keywords = [
    "environment", "environmental", "biodiversity", "nature", "ecosystem", "ecosystems",
    "species", "ocean", "oceans", "forest", "forests", "water", "deforest", "deforestation",
    "endangered", "protect", "protection", "agriculture", "agricultural", "continent",
    "hemisphere", "climate", "climatic", "warming", "global warming", "extreme weather",
    "cyclone", "cyclones", "biology", "biologist", "chemistry", "chemical", "chemicals",
    "sustainable", "sustainability", "sustain", "green", "greenwashing", "greenhouse",
    "renewable", "renewables", "energy", "clean energy", "net zero", "carbon neutrality",
    "carbon neutral", "transition", "transitions", "transitioning", "decarbon",
    "decarbonize", "decarbonise", "decarbonization", "decarbonisation",
    "alternative energy", "alternative fuel", "green investment", "green investments",
    "circular economy", "low carbon", "climate positive", "energy efficiency",
    "energy efficient", "dual challenge", "ambition", "ambitions", "commitment",
    "commitments", "leadership", "vision", "visionary", "carbon", "carbon dioxide", "co2",
    "ghg", "greenhouse gas", "greenhouse gases", "emission", "emissions", "methane", "ch4",
    "footprint", "carbon footprint", "effluent", "effluents", "pollutant", "pollutants",
    "pollution", "hazardous", "hazard", "contaminated", "contamination", "disposal",
    "waste disposal", "flaring", "gas flaring", "abatement", "carbon abatement",
    "carbon capture", "carbon sink", "carbon offset", "carbon offsets", "carbon tax",
    "carbon pricing", "carbon price", "solar", "solar power", "solar energy", "wind",
    "wind energy", "wind power", "hydrogen", "green hydrogen", "biofuel", "biofuels",
    "biomass", "battery", "batteries", "geothermal", "electric vehicle",
    "electric vehicles", "ev", "evs", "hydropower", "hydroelectric", "clean tech",
    "material", "materials", "metric", "metrics", "target", "targets", "tonnes", "tons",
    "scope 1", "scope 2", "scope 3", "baseline", "benchmark", "benchmarks", "reduction",
    "reductions", "increase", "increases", "ogci", "ipcc", "paris agreement", "paris",
    "kyoto", "unfccc", "1.5°c", "2°c", "two degrees", "one point five degrees"
]

# 5. Regex pattern
pattern = re.compile(
    r'\b(?:' + '|'.join(re.escape(word) for word in greenwashing_keywords) + r')\b',
    flags=re.IGNORECASE
)

# 6. Filter rows
df_filtered = df[df["Text"].apply(lambda x: bool(pattern.search(x)))]

# 7. Adding word counts and chr counts
df_filtered["Word_Count"] = df_filtered["Text"].str.split().str.len()
df_filtered["Char_Count"] = df_filtered["Text"].str.len()

# 8. Save
df_filtered.to_csv("Datasets/BIGOIL_Cleaned_Dataset.csv", index=False)

print(f"✅ Done. {len(df_filtered)} rows retained.")
print("✅ Saved: BIGOIL_Cleaned_Dataset.csv", "Rows:", len(df_filtered))
df_filtered

✅ Done. 7224 rows retained.
✅ Saved: BIGOIL_Cleaned_Dataset.csv Rows: 7224


/var/folders/_r/x1wl6kys6bbcy57xk9d0db3w0000gn/T/ipykernel_88720/1007914732.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Word_Count"] = df_filtered["Text"].str.split().str.len()
/var/folders/_r/x1wl6kys6bbcy57xk9d0db3w0000gn/T/ipykernel_88720/1007914732.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Char_Count"] = df_filtered["Text"].str.len()


,Company,Year,Page,Text,Char_Count,Word_Count,Is_Sparse,Subpage
0,SHELL,2022,1,responsible energy #poweringprogress shell plc...,73,8,False,1.0
1,SHELL,2022,2,contents read the shell sustainability report ...,1790,270,False,2.0
2,SHELL,2022,3,sustainability at shell welcome to the shell s...,510,76,False,3.0
3,SHELL,2022,4,letter from the ceo wael sawan chief executive...,2422,396,False,4.1
4,SHELL,2022,4,"we joined two major projects in qatar, for exa...",1757,280,False,4.2
...,...,...,...,...,...,...,...,...
9033,BP,2019,79,we have applied the international standard on ...,2240,324,False,79.3
9034,BP,2019,80,generation and storage; creating a diverse and...,2428,391,False,80.1
9035,BP,2019,80,"these statements may generally, but not always...",2297,354,False,80.2
9036,BP,2019,81,about this report order copies bp’s printed pu...,814,110,False,81.0


In [ ]:
# Adding the new character counts etc --> INTEGRATED ABOVE DW DELETE LATER 

df = pd.read_csv("Datasets/BIGOIL_Cleaned_Dataset.csv")

df["Word_Count"] = df["Text"].str.split().str.len()
df["Char_Count"] = df["Text"].str.len()

df.to_csv("/content/cleaned_oil_chunked_fixed.csv", index=False)
print("✅ Saved: cleaned_oil_chunked_fixed.csv", "Rows:", len(df))

In [21]:
# lemmantion etc
# 1. Load the CLEAN & CHUNKED dataset
df = pd.read_csv("Datasets/BIGOIL_Cleaned_Dataset.csv")

# 2. Load spaCy with only what we need
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

# 3. Lemmatisation function
def lemmatise_text(text):
    doc = nlp(str(text))
    lemmas = [
        token.lemma_
        for token in doc
        if not token.is_punct and not token.is_space
    ]
    return " ".join(lemmas)

# 4. Apply lemmatization
df["lemmatised_text"] = df["Text"].apply(lemmatise_text)

# 5. Save output
df.to_csv("Datasets/BIGOIL_Lemmatised_Dataset.csv", index=False)

print("✅ Lemmatisation complete. Saved as BIGOIL_Lemmatised_Dataset.csv")
df


✅ Lemmatisation complete. Saved as BIGOIL_Lemmatised_Dataset.csv


,Company,Year,Page,Text,Char_Count,Word_Count,Is_Sparse,Subpage,lemmatised_text
0,SHELL,2022,1,responsible energy #poweringprogress shell plc...,73,8,False,1.0,responsible energy poweringprogress shell plc ...
1,SHELL,2022,2,contents read the shell sustainability report ...,1790,270,False,2.0,content read the shell sustainability report o...
2,SHELL,2022,3,sustainability at shell welcome to the shell s...,510,76,False,3.0,sustainability at shell welcome to the shell s...
3,SHELL,2022,4,letter from the ceo wael sawan chief executive...,2422,396,False,4.1,letter from the ceo wael sawan chief executive...
4,SHELL,2022,4,"we joined two major projects in qatar, for exa...",1757,280,False,4.2,we join two major project in qatar for example...
...,...,...,...,...,...,...,...,...,...
7219,BP,2019,79,we have applied the international standard on ...,2240,324,False,79.3,we have apply the international standard on qu...
7220,BP,2019,80,generation and storage; creating a diverse and...,2428,391,False,80.1,generation and storage create a diverse and in...
7221,BP,2019,80,"these statements may generally, but not always...",2297,354,False,80.2,these statement may generally but not always b...
7222,BP,2019,81,about this report order copies bp’s printed pu...,814,110,False,81.0,about this report order copy bp ’s print publi...


In [2]:
# basic sentiment analysis

# 1. Load your latest dataset
df = pd.read_csv("Datasets/BIGOIL_Lemmatised_Dataset.csv")

# 2. Load RoBERTa-based sentiment model
sentiment_model = pipeline(
    "sentiment-analysis",
    model="siebert/sentiment-roberta-large-english"
)

# 3. Sentiment function (safe truncation)
def get_sentiment(text):
    result = sentiment_model(
        str(text),
        truncation=True
    )[0]
    return result["label"], float(result["score"])

# 4. Apply sentiment analysis
df[["sentiment_label", "sentiment_score"]] = df["Text"].apply(
    lambda x: pd.Series(get_sentiment(x))
)

# 5. Save output
output_path = "Datasets/BIGOIL_Sentiment_Dataset.csv"
df.to_csv(output_path, index=False)

print("✅ RoBERTa sentiment complete!")
print("Saved as:", output_path)

# Quick sanity check
print(df[["Text", "sentiment_label", "sentiment_score"]].head(5))


Device set to use mps:0


✅ RoBERTa sentiment complete!
Saved as: Datasets/BIGOIL_Sentiment_Dataset.csv
                                                Text sentiment_label  \
0  responsible energy #poweringprogress shell plc...        POSITIVE   
1  contents read the shell sustainability report ...        POSITIVE   
2  sustainability at shell welcome to the shell s...        POSITIVE   
3  letter from the ceo wael sawan chief executive...        POSITIVE   
4  we joined two major projects in qatar, for exa...        POSITIVE   

   sentiment_score  
0         0.998575  
1         0.997634  
2         0.998773  
3         0.998425  
4         0.998379  


In [ ]:
# =====================================================
# ClimateBERT Full Inference Pipeline (ALL MODELS)
# =====================================================

# =====================================================
# 3. Load dataset
# =====================================================

INPUT_PATH = "Datasets/BIGOIL_Sentiment_Dataset.csv"
OUTPUT_PATH = "Datasets/BIGOIL_Sentiment_ClimateBert_Dataset.csv"

df = pd.read_csv(INPUT_PATH)

print("Loaded dataset:", df.shape)

# =====================================================
# 4. Helper function to run any ClimateBERT classifier
# =====================================================

def run_climate_model(model_name, prefix, df, batch_size=16):
    """
    Runs a HuggingFace text-classification model on df['Text']
    and appends label, score, and full probability distribution.
    """

    print(f"\nRunning model: {model_name}")

    clf = pipeline(
        "text-classification",
        model=model_name,
        device=0,                 # GPU
        truncation=True,
        return_all_scores=True
    )

    labels = []
    scores = []
    probs  = []

    texts = df["Text"].astype(str).tolist()

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        outputs = clf(batch)

        for out in outputs:
            # Sort predictions by confidence
            out_sorted = sorted(out, key=lambda x: x["score"], reverse=True)
            best = out_sorted[0]

            labels.append(best["label"])
            scores.append(float(best["score"]))
            probs.append({d["label"]: float(d["score"]) for d in out})

    df[f"{prefix}_label"] = labels
    df[f"{prefix}_score"] = scores
    df[f"{prefix}_probs"] = probs

    print(f"✅ Finished: {prefix}")

# =====================================================
# 5. Run ALL ClimateBERT models
# =====================================================

# 5.1 Specificity (specific vs non-specific)
run_climate_model(
    model_name="climatebert/distilroberta-base-climate-specificity",
    prefix="specificity",
    df=df
)

# 5.2 Climate detector (climate vs not)
run_climate_model(
    model_name="climatebert/distilroberta-base-climate-detector",
    prefix="climate_detector",
    df=df
)

# 5.3 Commitment framing
run_climate_model(
    model_name="climatebert/distilroberta-base-climate-commitment",
    prefix="commitment",
    df=df
)

# 5.4 TCFD category classification
run_climate_model(
    model_name="climatebert/distilroberta-base-climate-tcfd",
    prefix="tcfd",
    df=df
)

# 5.5 Climate-specific sentiment
run_climate_model(
    model_name="climatebert/distilroberta-base-climate-sentiment",
    prefix="climate_sentiment",
    df=df
)

# =====================================================
# 6. Save final dataset
# =====================================================

df.to_csv(OUTPUT_PATH, index=False)

print("\n🎉 ALL CLIMATEBERT MODELS COMPLETED")
print("Saved to:", OUTPUT_PATH)
print("Final shape:", df.shape)


Loaded dataset: (7224, 11)

Running model: climatebert/distilroberta-base-climate-specificity


Device set to use mps:0
/Users/rezafayaz/Documents/Dissertation MDS /Final Files/BigOilNLPAnalysis/.venv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


✅ Finished: specificity

Running model: climatebert/distilroberta-base-climate-detector


Device set to use mps:0
/Users/rezafayaz/Documents/Dissertation MDS /Final Files/BigOilNLPAnalysis/.venv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


✅ Finished: climate_detector

Running model: climatebert/distilroberta-base-climate-commitment


Device set to use mps:0
/Users/rezafayaz/Documents/Dissertation MDS /Final Files/BigOilNLPAnalysis/.venv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


✅ Finished: commitment

Running model: climatebert/distilroberta-base-climate-tcfd


Device set to use mps:0
/Users/rezafayaz/Documents/Dissertation MDS /Final Files/BigOilNLPAnalysis/.venv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


✅ Finished: tcfd

Running model: climatebert/distilroberta-base-climate-sentiment


Device set to use mps:0
/Users/rezafayaz/Documents/Dissertation MDS /Final Files/BigOilNLPAnalysis/.venv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# addition of buzzword scoring and fuutre scoring

import pandas as pd
import spacy
import re

# Load your dataset
df = pd.read_csv('/content/clean_fresh_oil_with_all_climatebert.csv')

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# --- POS-based future verbs (as lemmas) ---
future_verbs = {
    "will", "shall", "aim", "aiming", "plan", "planning", "target", "aspire",
    "commit", "committed", "pledge", "pledged", "pledging", "vow", "promise",
    "seek", "intend", "intends", "hope", "envision", "anticipate", "anticipates", "forecast",
    "forecasting", "project", "projects", "projected", "look", "working", "moving", "set"
}

# --- Phrase-based future expressions (not captured as single tokens by spaCy) ---
future_phrases = [
    "going to", "seek to", "intend to", "set out to", "hope to", "looking to",
    "working to", "working towards", "moving towards", "transition plan",
    "path to net zero", "by 2030", "by 2050",
    "we’re helping to save the planet"
]

# --- Buzzword lexicon for greenwashing ---
greenwashing_lexicon = [
    "green", "clean", "cleaner", "cleanest", "efficient", "sustainable",
    "sustainability", "eco-friendly", "environmentally friendly", "earth-friendly",
    "eco-conscious", "natural", "non-toxic", "organic", "ethical", "biodegradable",
    "carbon neutral", "climate neutral", "carbon offsets", "carbon offset", "carbon credits",
    "low carbon", "fuels of tomorrow", "resilient hydrocarbons", "energy in progress",
    "transformation", "energy transition", "transition", "beyond petroleum",
    "emissions intensity", "locally grown", "sustainably sourced", "eco-safe",
    "eco-preferred", "cfc-free", "chlorofluorocarbon-free", "renewable natural gas", "rng",
    "carbon capture and storage", "ccs", "e-fuels", "synthetic fuels", "synth-fuels",
    "carbon-neutral e-fuels", "drive carbon neutral"
]

# --- Compile regex patterns for phrase matching ---
future_phrase_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(phrase) for phrase in future_phrases) + r')\b', flags=re.IGNORECASE)
buzzword_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(term) for term in greenwashing_lexicon) + r')\b', flags=re.IGNORECASE)

# --- POS-based future score function ---
def pos_future_score(text):
    if pd.isnull(text) or text.strip() == "":
        return 0.0

    doc = nlp(text)
    token_count = len(doc)
    future_count = 0

    for i, token in enumerate(doc):
        lemma = token.lemma_.lower()
        pos = token.pos_

        # Match modal verbs like "will", "shall" only when followed by a verb
        if lemma in {"will", "shall"} and token.tag_ == "MD":
            if i + 1 < token_count and doc[i + 1].pos_ == "VERB":
                future_count += 1
            else:
                future_count += 1  # accept anyway to stay inclusive

        # Match intention/commitment words when used as verbs or auxiliaries
        elif lemma in future_verbs and pos in {"VERB", "AUX"}:
            future_count += 1

    # Also match multi-word future phrases via regex
    future_count += len(future_phrase_pattern.findall(text.lower()))

    return future_count / token_count if token_count > 0 else 0.0

# --- Buzzword score function (greenwashing density) ---
def buzzword_score(text):
    if pd.isnull(text) or text.strip() == "":
        return 0.0
    word_count = len(text.split())
    if word_count == 0:
        return 0.0
    buzzword_matches = buzzword_pattern.findall(text.lower())
    return len(buzzword_matches) / word_count

# --- Apply both scoring functions to your DataFrame ---
df["future_score_pos"] = df["Text"].apply(pos_future_score)
df["buzzword_score"] = df["Text"].apply(buzzword_score)

# --- Save new dataset with both scores included ---
df.to_csv("/content/oil_reports_with_future_and_buzzword_scores.csv", index=False)

print("✅ POS-based future score and buzzword score added!")
print("📄 Saved as: oil_reports_with_future_and_buzzword_scores.csv")
